IMPORTS

In [ ]:
import flodym as fd
import flodym.export as fde
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flodym import StockArray

PARAMETER DEFINITION

In [ ]:
parameter_definitions = [
    ParameterDefinition(name="pv_future_capacity", dim_letters=("t",)),
    ParameterDefinition(name="battery_future_capacity", dim_letters=("t",)),
    ParameterDefinition(name="pv_material_annual_decline", dim_letters=("e","p")),
    ParameterDefinition(name="pv_initial_composition", dim_letters=("e","p")),
    ParameterDefinition(name="battery_material_annual_decline", dim_letters=("e","p")),
    ParameterDefinition(name="battery_initial_composition", dim_letters=("e","p")),
]

DIMENSION DEFINITION

In [ ]:
Future_time = fd.Dimension(letter="f", name="Future Time", dtype=int, items=list(range(2025, 2046)))
Whole_time = fd.Dimension(letter="w", name="Whole Time", dtype=int, items=list(range(2005, 2046)))
Historic_time = fd.Dimension(letter="h", name="Historic Time", dtype=int, items=list(range(2005, 2025)))
product = fd.Dimension(letter = "p", name = "Product", dtype=str,  items=["PV", "Battery"])
element = fd.Dimension(letter = "e", name = "Materials PV", dtype=str,  items=["Si", "Al", "Cu", "Ag", "Li", "Steel", "LiPF6", "PP"])
dims = fd.DimensionSet(dim_list=[Future_time, Whole_time, Historic_time, product, element])


In [ ]:
PV = np.array([110,131,152,173,194,215,234,253,272,291,310,328,346,364,382,400,416,432,448,464,480])

Batt = np.array([27.50,37.17,48.25,60.75,74.67,90.00,104.10,119.20,135.30,152.40,170.50,190.24,211.06,232.96,255.94,280.00,303.68,328.32,353.92,380.48,408.00
])# put them in a file and call them in parameter

CREATING HISTORIC STOCKS

In [ ]:
prescribed_stock =StockArray(dims=dims["f","p"])
prescribed_stock["PV"].values[...] = PV
prescribed_stock["Battery"].values[...] = Batt

slope = (prescribed_stock.values[1,:] - prescribed_stock.values[0,:]) / (dims["f"].items[1] - dims["f"].items[0])
diff_to_first_future_year = dims["f"].items[0] - np.array(dims["h"].items)
values = prescribed_stock.values[0,:] - slope[np.newaxis,:] * diff_to_first_future_year[:,np.newaxis]
values = np.maximum(values, 0)  # avoid negative stock
runup_stock = fd.FlodymArray(
    dims=dims["h","p"],
    values = values,
)

LIFETIME MODEL

In [ ]:
lifetime_mean = fd.FlodymArray(dims=dims["p",])
lifetime_mean["PV"] = 20.
lifetime_mean["Battery"] = 10.
    
lifetime_std = fd.FlodymArray(dims=dims["p",])
lifetime_std["PV"] = 4.
lifetime_std["Battery"] = 2.

lifetime = fd.lifetime_models.LogNormalLifetime(dims=dims["w","p"], mean=lifetime_mean, std=lifetime_std,   time_letter="w",)

STOCK_DRIVEN_MODEL

In [ ]:
dsm_sdriv = fd.StockDrivenDSM(
    dims=dims["w","p"],
    lifetime_model=lifetime,
    time_letter="w",
)
dsm_sdriv.stock[{"w": dims["h"]}] = runup_stock
dsm_sdriv.stock[{"w": dims["f"]}] = prescribed_stock

dsm_sdriv.compute()  

# Stock (whole period)
ptr = fde.PlotlyArrayPlotter(
    array=dsm_sdriv.stock,
    intra_line_dim="Whole Time",
    title="Stock (whole period)"
)
fig = ptr.plot(do_show=True)

# Stock (whole period)
ptr = fde.PlotlyArrayPlotter(
    array=dsm_sdriv.inflow,
    intra_line_dim="Whole Time",
    title="Inflow (whole period)"
)
fig = ptr.plot(do_show=True)


# Relevant part of inflow
ptr = fde.PlotlyArrayPlotter(
    array=flows["manufacturing => use"],
    intra_line_dim="Future Time",
    title="Adjacent flow (future period only)"
)
fig = ptr.plot(do_show=True)


In [ ]:
class SimpleMFA(MFASystem):
    def compute(self):
        t=fd.FlodymArray(dims=self.dims["t",], values=np.array(self.dims["t"].items)) - self.dims["t"].items[0]
        eta=(1 -  (self.parameters["material_annual_decline"]/100))**t
        
        self.flows["sysenv => manufacturing"][...] = (
            self.parameters["initial composition"] * eta * (self.parameters["cap_batteries"])
        )
        
        self.flows["manufacturing => use"][...] = self.flows["use => sysenv"]

        # use stock
        self.stocks["use"].inflow[...] = self.flows["manufacturing => use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=10,
            std=2,
        )
        self.stocks["use"].compute()

        # end-of-life  flow
        self.flows["use => sysenv"][...] = self.stocks["use"].outflow

In [ ]:
dsm_idriv = fd.InflowDrivenDSM(
    dims=dims["w","p"],
    lifetime_model=dsm_sdriv.lifetime_model,
    time_letter="w",
)
dsm_idriv.inflow[...] = dsm_sdriv.inflow * material_composition * eta # eta has dimension w


# The dictionary in the MFA system will look somewhat like this
flows = {
    "use => eol": fd.FlodymArray(dims=dims["f","p"], name="use => eol"),
    "manufacturing => use": fd.FlodymArray(dims=dims["f","p"], name="manufacturing => use"),
}
# Copy only the future part of the inflows and outflows to the surrounding flows
flows["use => eol"][...] = dsm_idriv.outflow[{"w","p": dims["f","p"]}]
flows["manufacturing => use"][...] = dsm_idriv.inflow[{"w","p": dims["f","p"]}]
